In [ ]:
#from torch_geometric.datasets import QM9
import pickle
import pandas as pd

In [ ]:
from tqdm.notebook import tqdm, trange

In [ ]:
import torch
torch.manual_seed(3)
from torch_geometric.nn import SchNet

In [ ]:
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, radius_graph
from torch.nn import Embedding, Linear, ModuleList, Sequential, MSELoss

In [ ]:
with open("train_struct_pyg.pickle", 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
import random

random.seed(34)

In [ ]:
formation_energys = [i.formation_energy_ev_natom for i in dataset]

In [ ]:
min(formation_energys)

In [ ]:
def train(target_field: str = 'bandgap_energy_ev'):
    """
    target_field = ['formation_energy_ev_natom', 'bandgap_energy_ev']
    """
    import pdb; pdb.set_trace()
    schnet = SchNet()
    mseloss = MSELoss()
    optimizer = torch.optim.Adam(schnet.parameters(), lr=1e-3, weight_decay=5e-4)
    train_data = dataset[:2000]
    val_data = dataset[2001:]
    for epoch in range(5):
        schnet.train(True)
        random.shuffle(train_data)
        train_losses = []
        for data in tqdm(train_data, desc="Train"):    
            optimizer.zero_grad()
            out = schnet(data.z, data.xyz)
            loss = mseloss(out, torch.FloatTensor([data[target_field]]))
            loss.backward()
            optimizer.step()
            train_losses.append(loss)
        print(f"Train loss: {torch.mean(torch.stack(train_losses), dim=0)}")
        schnet.train(False)
        val_losses = []
        for data in tqdm(val_data, desc="Test"):
            out = schnet(data.z, data.xyz)
            loss = mseloss(out,torch.FloatTensor([data[target_field]]))
            val_losses.append(loss)
        print(f"Val loss: {torch.mean(torch.stack(val_losses), dim=0)}")
        torch.save(schnet, f"{target_field}/schnet_epoch_{epoch}")

## `bandgap_energy_ev`

In [ ]:
train('formation_energy_ev_natom')

In [ ]:
train('bandgap_energy_ev')

In [ ]:
# todo prunning

config = {
    "hidden_channels": tune.randint(64, 256),
    "num_filters": tune.randint(64, 256),
    "num_interactions": tune.randint(4, 16),    
    "momentum": tune.uniform(0.1, 0.9),
    "num_gaussians": tune.randint(16,64),
    "cutoff": tune.uniform(1., 16.) 
}

scheduler = ASHAScheduler(
        max_t=16,
        grace_period=1,
        reduction_factor=2)

result = tune.run(
    tune.with_parameters(train),
    resources_per_trial={"cpu": 1},
    config=config,
    metric="loss",
    mode="min",
    num_samples=num_samples,
    scheduler=scheduler
)


- hidden_channels: int = 128, 
- num_filters: int = 128,               
- num_interactions: int = 6, 
- num_gaussians: int = 50,
- cutoff: float = 10.0, 
- max_num_neighbors: int = 32,
- readout: str = 'add', or 'mean'
- dipole: bool = False,
- mean: Optional[float] = None  The mean of the property to predict
- std: Optional[float] = None  The std of the property to predict

## Load model and run test

In [ ]:
with open("test_struct_pyg.pickle", 'rb') as f:
    test = pickle.load(f)
len(test)

In [ ]:
filename_bandgap_energy_ev = 'bandgap_energy_ev/schnet_epoch_3'
filename_formation_energy_ev_natom = 'formation_energy_ev_natom/schnet_epoch_3'

In [ ]:
with open(filename_bandgap_energy_ev, 'rb') as f:
    model = torch.load(f)

bandgap_energy_ev = []
for data in tqdm(test):
    bandgap_energy_ev.append(model(data.z, data.xyz).item())

In [ ]:
with open(filename_formation_energy_ev_natom, 'rb') as f:
    model = torch.load(f)

formation_energy_ev_natom = []
for data in tqdm(test):
    formation_energy_ev_natom.append(model(data.z, data.xyz).item())

In [ ]:
test_df = pd.read_csv("test.csv")

In [ ]:
test_df['bandgap_energy_ev'] = bandgap_energy_ev
test_df['formation_energy_ev_natom'] = formation_energy_ev_natom

In [ ]:
test_df = test_df[['id', 'bandgap_energy_ev', 'formation_energy_ev_natom']]

In [ ]:
test_df = test_df.clip(lower=0)

In [ ]:
test_df.to_csv("answer.csv", index=False)